In [41]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
from tqdm.notebook import tnrange

In [42]:
def solve_linear(a, b, c, d):
    nf = len(d)
    ac, bc, cc, dc = map(np.array, (a, b, c, d))
    for it in range(1, nf):
        mc = ac[it - 1] / bc[it - 1]
        bc[it] = bc[it] - mc * cc[it - 1] 
        dc[it] = dc[it] - mc * dc[it - 1]
    xc = bc
    xc[-1] = dc[-1] / bc[-1]

    for il in range(nf - 2, -1, -1):
        xc[il] = (dc[il] - cc[il] * xc[il + 1]) / bc[il]

    return xc

In [43]:
def get_dt(dx, u, chi, coeff=0.9):
    return coeff * (dx ** 2) / (u * dx + 2 * chi)

In [75]:
u = 0.
chi = 8e-12
dx = 4e-7
dt = 0.01

s = u * dt / dx
r = chi * dt / (dx ** 2)

In [76]:
print(r)

0.5000000000000001


In [77]:
teta = 1
a = 0
b = 1e-4

In [78]:
K = int((b - a) / dx) + 1

In [79]:
start = np.zeros(K, dtype=np.float64)
for i in range(K // 2):
    start[i] = 1.

In [80]:
def left_border(t):
    return 0
    
def right_border(t):
    return 0

In [81]:
def solve(solver):
    t_iterations = int(teta / dt) + 1

    Ts = []
    Ts.append(start)
    
    for cur_t_iter in tnrange(1, t_iterations + 1):
        t = dt * cur_t_iter
        cur = solver(t, Ts)
        Ts.append(cur)
    
    return Ts

In [90]:
def evident_opposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t + r * (prev_t + next_t - 2 * cur_t)
    
    return cur

In [83]:
def evident_nonopposite(t, Ts):
    cur = np.zeros(K, dtype=np.float64)

    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-1][i]

        cur[i] = cur_t - s * (next_t - cur_t) + r * (prev_t + next_t - 2 * cur_t)
    
    return cur

In [84]:
def confusion(t, Ts):
    assert math.isclose(chi, 0.)
    
    if len(Ts) == 1:
        return evident_opposite(t, Ts)
    
    cur = np.zeros(K, dtype=np.float64)
    
    for i in range(K):
        prev_t = Ts[-1][i - 1] if i > 0 else left_border(t)
        next_t = Ts[-1][i + 1] if i < K - 1 else right_border(t)
        cur_t = Ts[-2][i]

        cur[i] = cur_t - s * (next_t - prev_t)
    
    return cur

In [85]:
def nonevident_opposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)
    
    for i in range(K):
        b[i] = 1 + s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * (r + s)
    d[K - 1] += right_border(t) * r
    for i in range(K - 1):
        a[i] = - r - s
        c[i] = -r
    return solve_linear(a, b, c, d)

In [86]:
def nonevident_nonopposite(t, Ts):
    a = np.zeros(K - 1, dtype=np.float64)
    b = np.zeros(K, dtype=np.float64)
    c = np.zeros(K - 1, dtype=np.float64)
    d = np.zeros(K, dtype=np.float64)
    
    for i in range(K):
        b[i] = 1 - s + 2 * r
        d[i] = Ts[-1][i]
    d[0] += left_border(t) * r
    d[K - 1] -= right_border(t) * (s - r)
    
    for i in range(K - 1):
        a[i] = -r
        c[i] = s - r
    
    return solve_linear(a, b, c, d)

In [91]:
Ts = solve(evident_opposite)

In [92]:
print(Ts[80])

[8.89278788e-02 1.75686785e-01 2.62445691e-01 3.43007532e-01
 4.23569374e-01 4.94763559e-01 5.65957745e-01 6.25825582e-01
 6.85693420e-01 7.33587690e-01 7.81481961e-01 8.17923253e-01
 8.54364546e-01 8.80726332e-01 9.07088118e-01 9.25211846e-01
 9.43335574e-01 9.55171478e-01 9.67007382e-01 9.74345642e-01
 9.81683902e-01 9.86000526e-01 9.90317150e-01 9.92724498e-01
 9.95131846e-01 9.96403652e-01 9.97675459e-01 9.98311362e-01
 9.98947265e-01 9.99247874e-01 9.99548483e-01 9.99682683e-01
 9.99816883e-01 9.99873389e-01 9.99929894e-01 9.99952302e-01
 9.99974709e-01 9.99983064e-01 9.99991419e-01 9.99994344e-01
 9.99997268e-01 9.99998227e-01 9.99999186e-01 9.99999480e-01
 9.99999773e-01 9.99999857e-01 9.99999941e-01 9.99999964e-01
 9.99999986e-01 9.99999991e-01 9.99999997e-01 9.99999998e-01
 9.99999999e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.000000

In [93]:
xs = [
    a + i * dx
    for i in range(K)
]

y_min = np.min(np.array(Ts))
y_max = np.max(np.array(Ts))

def plot_img(cur_index):
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(xs, Ts[cur_index])

    ax.set_ylim(y_min - 0.1, y_max + 0.1)

    fig.canvas.draw()
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    plt.close(fig)

    return image

frames = [
    plot_img(i)
    for i in tqdm_notebook(range(len(Ts)))
    if i % 10 == 0
]
imageio.mimsave('./T.gif', frames, fps=24)

c:\users\hanve\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
